## Import libraries

In [1]:
import requests
from bs4 import BeautifulSoup
import sqlite3

conn = sqlite3.connect('books.db')
c = conn.cursor()


def insert_data_in_db(sql, data):
    global conn, c
    c.execute(sql, data)
    conn.commit()
    
    
def clear_database():
    global conn, c
    c.execute("DELETE FROM books")
    c.execute("DELETE FROM categories")
    c.execute("DELETE FROM sqlite_sequence")
    conn.commit()

In [2]:
response = requests.request('GET', 'http://books.toscrape.com/catalogue/category/books/mystery_3/index.html')
soup = BeautifulSoup(response.content, 'html.parser')

In [3]:
all_articles = soup.find_all('article', attrs={'class': 'product_pod'})
len(all_articles)

20

In [4]:
all_articles[0]

<article class="product_pod">
<div class="image_container">
<a href="../../../sharp-objects_997/index.html"><img alt="Sharp Objects" class="thumbnail" src="../../../../media/cache/32/51/3251cf3a3412f53f339e42cac2134093.jpg"/></a>
</div>
<p class="star-rating Four">
<i class="icon-star"></i>
<i class="icon-star"></i>
<i class="icon-star"></i>
<i class="icon-star"></i>
<i class="icon-star"></i>
</p>
<h3><a href="../../../sharp-objects_997/index.html" title="Sharp Objects">Sharp Objects</a></h3>
<div class="product_price">
<p class="price_color">£47.82</p>
<p class="instock availability">
<i class="icon-ok"></i>
    
        In stock
    
</p>
<form>
<button class="btn btn-primary btn-block" data-loading-text="Adding..." type="submit">Add to basket</button>
</form>
</div>
</article>

In [5]:
price = all_articles[0].find('p', attrs={'class': 'price_color'})
print(float(price.get_text().split('£')[1]))

book_name = all_articles[0].find('h3').find('a').get('title')
print(book_name)

rating = all_articles[12].find('p').get('class')[1]
print(rating)

47.82
Sharp Objects
Three


## Make 2 functions
**one for getting all categories names and urls and another one for scrapping books for each category url**

In [6]:
def get_all_books_categories_links(link):
    categories = {}
    response = requests.request('GET', link)
    soup = BeautifulSoup(response.content, 'html.parser')
    all_book_links = soup.find('div', attrs={'class': 'side_categories'}).find('ul').find('li').find('ul').find_all('a')
    for book_link in all_book_links:
        true_url = book_link.get('href').replace('../', 'http://books.toscrape.com/catalogue/category/books/')
        if true_url != 'index.html':
            category_name = book_link.get_text().replace(' ', '').replace('\n', '')
            categories[category_name] = true_url
    return categories

In [7]:
def get_books(page, category_id):
    rates = {'One': 1, 'Two': 2, 'Three': 3, 'Four': 4, 'Five': 5}
    response = requests.request('GET', page)
    soup = BeautifulSoup(response.content, 'html.parser')
    all_articles = soup.find_all('article', attrs={'class': 'product_pod'})
    for article in all_articles:
        price = float(article.find('p', attrs={'class': 'price_color'}).get_text().split('£')[1])
        book_name = article.find('h3').find('a').get('title')
        rating = article.find('p').get('class')[1]
        sql = "INSERT INTO books (title,price,rate,category_id) VALUES (?,?,?,?)"
        data = [book_name,price,rates[rating],category_id]
        insert_data_in_db(sql, data)
        print(f'Book name: {book_name}\nPrice: {price}\nRate: {rates[rating]}')
        print('-------------------------------')

## Start Scrapping and adding data to Database

In [ ]:
clear_database()
categories = get_all_books_categories_links('http://books.toscrape.com/catalogue/category/books/travel_2/index.html')
for category_name, url in categories.items():
    print('++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++')
    print(f'+ For link: {url}, category name: [{category_name}]')
    print('++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++\n')
    insert_data_in_db("INSERT INTO categories (name) VALUES (?)", [category_name])
    get_books(url, c.lastrowid)
    print('\n\n\n')
conn.close()

++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
+ For link: http://books.toscrape.com/catalogue/category/books/mystery_3/index.html, category name: [Mystery]
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

Book name: Sharp Objects
Price: 47.82
Rate: 4
-------------------------------
Book name: In a Dark, Dark Wood
Price: 19.63
Rate: 1
-------------------------------
Book name: The Past Never Ends
Price: 56.5
Rate: 4
-------------------------------
Book name: A Murder in Time
Price: 16.64
Rate: 1
-------------------------------
Book name: The Murder of Roger Ackroyd (Hercule Poirot #4)
Price: 44.1
Rate: 4
-------------------------------
Book name: The Last Mile (Amos Decker #2)
Price: 54.21
Rate: 2
-------------------------------
Book name: That Darkness (Gardiner and Renner #1)
Price: 13.92
Rate: 1
-------------------------------
Book name: Tastes Like Fear (DI Marnie Rome #3)
Price: 10.6

Book name: Beowulf
Price: 38.35
Rate: 2
-------------------------------
Book name: And Then There Were None
Price: 35.01
Rate: 2
-------------------------------
Book name: The Story of Hong Gildong
Price: 43.19
Rate: 4
-------------------------------
Book name: The Little Prince
Price: 45.42
Rate: 2
-------------------------------
Book name: Sense and Sensibility
Price: 37.46
Rate: 1
-------------------------------
Book name: Of Mice and Men
Price: 47.11
Rate: 2
-------------------------------
Book name: Emma
Price: 32.93
Rate: 2
-------------------------------
Book name: Alice in Wonderland (Alice's Adventures in Wonderland #1)
Price: 55.53
Rate: 1
-------------------------------




++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
+ For link: http://books.toscrape.com/catalogue/category/books/philosophy_7/index.html, category name: [Philosophy]
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

Book name: I Am Pilgrim (Pilgrim #1)
Price: 10.6
Rate: 4
-------------------------------
Book name: Eligible (The Austen Project #4)
Price: 27.09
Rate: 3
-------------------------------




++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
+ For link: http://books.toscrape.com/catalogue/category/books/childrens_11/index.html, category name: [Childrens]
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

Book name: Birdsong: A Story in Pictures
Price: 54.64
Rate: 3
-------------------------------
Book name: The Bear and the Piano
Price: 36.89
Rate: 1
-------------------------------
Book name: The Secret of Dreadwillow Carse
Price: 56.13
Rate: 1
-------------------------------
Book name: The White Cat and the Monk: A Retelling of the Poem “Pangur Bán”
Price: 58.08
Rate: 4
-------------------------------
Book name: Little Red
Price: 13.47
Rate: 3
-------------------------------
Book name: Walt Dis

Book name: The Coming Woman: A Novel Based on the Life of the Infamous Feminist, Victoria Woodhull
Price: 17.93
Rate: 3
-------------------------------
Book name: The Boys in the Boat: Nine Americans and Their Epic Quest for Gold at the 1936 Berlin Olympics
Price: 22.6
Rate: 4
-------------------------------
Book name: Starving Hearts (Triangular Trade Trilogy, #1)
Price: 13.99
Rate: 2
-------------------------------
Book name: America's Cradle of Quarterbacks: Western Pennsylvania's Football Factory from Johnny Unitas to Joe Montana
Price: 22.5
Rate: 3
-------------------------------
Book name: Aladdin and His Wonderful Lamp
Price: 53.13
Rate: 3
-------------------------------
Book name: Penny Maybe
Price: 33.29
Rate: 3
-------------------------------
Book name: Maude (1883-1993):She Grew Up with the country
Price: 18.02
Rate: 2
-------------------------------
Book name: The Inefficiency Assassin: Time Management Tactics for Working Smarter, Not Longer
Price: 20.59
Rate: 5
-----------

Book name: A Court of Thorns and Roses (A Court of Thorns and Roses #1)
Price: 52.37
Rate: 1
-------------------------------
Book name: Throne of Glass (Throne of Glass #1)
Price: 35.07
Rate: 3
-------------------------------
Book name: The Glittering Court (The Glittering Court #1)
Price: 44.28
Rate: 1
-------------------------------
Book name: Hollow City (Miss Peregrine’s Peculiar Children #2)
Price: 42.98
Rate: 1
-------------------------------
Book name: The Star-Touched Queen
Price: 46.02
Rate: 5
-------------------------------
Book name: The Hidden Oracle (The Trials of Apollo #1)
Price: 52.26
Rate: 2
-------------------------------
Book name: The Bane Chronicles (The Bane Chronicles #1-11)
Price: 44.73
Rate: 4
-------------------------------
Book name: Island of Dragons (Unwanteds #7)
Price: 29.65
Rate: 1
-------------------------------
Book name: Demigods & Magicians: Percy and Annabeth Meet the Kanes (Percy Jackson & Kane Chronicles Crossover #1-3)
Price: 37.51
Rate: 5
------

Book name: The Collected Poems of W.B. Yeats (The Collected Works of W.B. Yeats #1)
Price: 15.42
Rate: 5
-------------------------------
Book name: The Crossover
Price: 38.77
Rate: 4
-------------------------------
Book name: Booked
Price: 17.49
Rate: 5
-------------------------------
Book name: Twenty Love Poems and a Song of Despair
Price: 30.95
Rate: 4
-------------------------------




++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
+ For link: http://books.toscrape.com/catalogue/category/books/paranormal_24/index.html, category name: [Paranormal]
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

Book name: Vampire Knight, Vol. 1 (Vampire Knight #1)
Price: 15.4
Rate: 1
-------------------------------




++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
+ For link: http://books.toscrape.com/catalogue/category/books/art_25/index.html, categ

# Great Job!